# Amateur Hour - Predicting Stocks using LightGBM
### Starter Kernel by ``Magichanics`` 
*([GitHub](https://github.com/Magichanics) - [Kaggle](https://www.kaggle.com/magichanics))*

This is more of an improvement in organization and efficiency compared to my previous notebook. Feel free to post suggestions or criticisms!

## Table of Contents

* [Step 1. Merging Datasets](#section1)
* [Step 2. Feature Engineering](#section2)
* [Step 3. Modelling using LightGBM](#section3)
* [Step 4. Applying the Model](#section4)

In [1]:
import numpy as np
import pandas as pd
import os
import gc
from itertools import chain

import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# import environment for data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


In [4]:
(market_train_df, news_train_df) = env.get_training_data()
sampling = False
if sampling:
    market_train_df = market_train_df.tail(400_000)
    news_train_df = news_train_df.tail(1_000_000)

<a id='section1'></a>
## Step 1. Merging Datasets

While most of the notebooks focuses only on the market dataset, I'm going to attempt on bringing both the news and market dataset together.

In [5]:
market_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0


In [6]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",1438,1,,False,11,275,"{'0857.HK', '0857.F', '0857.DE', 'PTR.N'}",PetroChina Co Ltd,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",4413,1,FEATURE,False,55,907,{'STA.N'},Travelers Companies Inc,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",2108,2,PRESS DIGEST,False,15,388,"{'WMT.DE', 'WMT.N'}",Wal-Mart Stores Inc,14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,"{'GOOG.O', 'GOOG.OQ', 'GOOGa.DE'}",Google Inc,13,0.149071,-1,0.752917,0.162715,0.084368,83,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,{'XMSR.O'},XM Satellite Radio Holdings Inc,11,0.149071,-1,0.699274,0.209360,0.091366,102,0,0,0,0,0,0,0,0,0,0


In [7]:
from datetime import datetime, timedelta
start = datetime(2015, 1, 1, 0, 0, 0).date()
market_train_df = market_train_df.loc[market_train_df['time'].dt.date >= start].reset_index(drop=True)
news_train_df = news_train_df.loc[news_train_df['time'].dt.date >= start].reset_index(drop=True)

### Cleaning Data
We will be removing the rows with the following qualities, as well as features that have almost no correlation with the given data:
* Empty headlines
* Repeat headlines
* Urgency of 2

In [8]:
def clean_data(market_df, news_df, train=True):
    
    # get rid of invalid rows
    news_df = news_df[news_df.headline != '']
    news_df = news_df[news_df.urgency != 2]
    
    # remove duplicate headlines with the same assetCodes
    news_df = news_df.drop_duplicates(subset=['assetCodes', 'headline'],keep='first')
    
    # keep only the following features
#     news_df = news_df[['time', 'assetCodes', 'headline', 'subjects', 'audiences']]
    news_df = news_df[['time', 'assetCodes', 'headline']]
    valid_market_cols = ['time', 'assetCode', 'volume', 'close', 'open',\
                           'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',\
                           'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
    if train:
        valid_market_cols += ['returnsOpenNextMktres10']
    market_df = market_df[valid_market_cols]

    return market_df, news_df

In [9]:
market_train_df, news_train_df = clean_data(market_train_df, news_train_df, train=True)

In [10]:
news_train_df.shape

(1773344, 3)

In [11]:
market_train_df.shape

(892200, 10)

### Expanding News data
We are going to be splitting the news data by assetCode.

In [12]:
def expanding_news(news_df):
    
    # split to list
    news_output = news_df.copy()
    news_output['assetCodes'] = news_output['assetCodes'].str.findall(f"'([\w\./]+)'")
    
    # separate to assetcodes
    assetCodes_expanded = list(chain(*news_output['assetCodes']))
    assetCodes_index = news_df.index.repeat(news_output['assetCodes'].apply(len))
    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
    
    # merge to dataframe
    merging_cols = [f for f in news_output if f not in ['assetCodes', 'sourceId']]
    news_df_expanded = pd.merge(df_assetCodes, news_output[merging_cols], left_on='level_0', 
                                right_index=True, suffixes=(['','_old']))
    
    return news_df_expanded

In [13]:
expand_train_df = expanding_news(news_train_df)

In [14]:
expand_train_df.tail()

,level_0,assetCode,time,headline
3243658,2054221,SGEN.O,2016-12-30 21:57:00+00:00,EQUITY ALERT: Rosen Law Firm Announces Investi...
3243659,2054221,SGEN.OQ,2016-12-30 21:57:00+00:00,EQUITY ALERT: Rosen Law Firm Announces Investi...
3243660,2054222,IPDN.O,2016-12-30 21:58:53+00:00,PROFESSIONAL DIVERSITY NETWORK INC - FILES FOR...
3243661,2054222,IPDN.OQ,2016-12-30 21:58:53+00:00,PROFESSIONAL DIVERSITY NETWORK INC - FILES FOR...
3243662,2054223,JFC.N,2016-12-30 22:00:00+00:00,"JPMorgan China Region Fund, Inc. Board to Subm..."


In [15]:
market_train_df.tail()

,time,assetCode,volume,close,open,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10
892195,2016-12-30 22:00:00+00:00,ZIOP.O,1608829.0,5.35,5.37,-0.165367,-0.138042,-0.139597,-0.135913,0.051189
892196,2016-12-30 22:00:00+00:00,ZLTQ.O,347830.0,43.52,43.62,0.002996,0.002989,0.008213,0.003210,-0.048555
892197,2016-12-30 22:00:00+00:00,ZNGA.O,7396601.0,2.57,2.58,-0.091873,-0.078571,-0.077252,-0.077188,0.011703
892198,2016-12-30 22:00:00+00:00,ZTO.N,3146519.0,12.07,12.50,-0.065066,-0.042146,-0.078104,-0.043813,0.083367
892199,2016-12-30 22:00:00+00:00,ZTS.N,1701204.0,53.53,53.64,0.023127,0.028177,0.026566,0.028719,-0.016220


### Cleaning Headlines
The following will simplify strings to only get the necessary words needed for text processing.

In [16]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re

ps = PorterStemmer()
sw = stopwords.words('english')

# this takes up a lot of time, so apply it when getting coefficients to filter out words.
def clean_headlines(headline):
    
    # remove numerical and convert to lowercase
    headline =  re.sub('[^a-zA-Z]',' ',headline)
    headline = headline.lower()
    
    # use stemming to simplify words
    headline_words_rough = headline.split(' ')
    
    # check if stopwords are present in headlines
    headline_words = []
    for word in headline_words_rough:
        if word not in sw:
            # use stemming to simplify
            headline_words.append(ps.stem(word))
    
    # join sentence back again
    return ' '.join(headline_words)

### Categorical Groupby
This will merge groups of categorical data together into either lists or sets.

In [17]:
def categorical_groupby(expand_df):

    # get categorical groupbys
    main_cols = ['time', 'assetCode']
    expand_headline_groupby = expand_df[main_cols + ['headline']].groupby(['time', 'assetCode'])
    #expand_cat_groupby = expand_df[main_cols + ['subjects', 'audiences']].groupby(['time', 'assetCode'])
    
    # split subjects and audiences
    def cat_to_list(x):
        try:
            if x.name not in ['time', 'assetCode'] and x.name != 'headline':
                result = []
                for item in x:
                    result += item
                return list(set(result)) # returns unique audiences/subjects
            elif x.name == 'headline':
                return list(x)
        except ValueError:
            return np.nan
    
    # convert groupby to dataframes
#     expand_cat_df = expand_cat_groupby.transform(lambda x: cat_to_list(x))
    expand_headline_df = expand_headline_groupby.transform(lambda x: cat_to_list(x)) # can't iterate through?

    # merge to categorical dataframes
#     return pd.concat([expand_cat_df, expand_headline_df], axis=1)
    return expand_headline_df
    

### Merge by time &  assetCode to News Article
We will be merging rows with the same time and assetCode.

In [18]:
def get_matches(market_df, expand_df):
    
    # get temporary columns as data
    temp_market_df = market_df[['time', 'assetCode']].copy()
    temp_expand_df = expand_df[['time', 'assetCode']].copy()
    
    # get indecies
    temp_expand_df['expand_index'] = temp_expand_df.index.values
    
    # join the two
    temp_expand_df.set_index(['time', 'assetCode'], inplace=True)
    temp_expand_market_df = temp_market_df.join(temp_expand_df, on=['time', 'assetCode'])
    
    # remove nulls
    temp_expand_market_df = temp_expand_market_df[temp_expand_market_df.expand_index.isnull() == False]
    expand_indicies = temp_expand_market_df['expand_index'].tolist()
    
    # do final cleanup
    del temp_market_df
    del temp_expand_df
    
    # fetch matches
    return expand_df.loc[expand_indicies]

def merge_by_code(market_df, expand_df):
    
    # use a copy
    market_df_copy = market_df.copy()
    
    # get expansion of rows
    expand_df = get_matches(market_df, expand_df)
    
    # check if empty
    if len(expand_df) == 0:
        
#         news_cols = ['subjects', 'audiences', 'headline']
        news_cols = ['headline']
        
        # create blank columns of what was supposed to be there
        for f in news_cols:
            market_df_copy[f] = np.nan
        
        return market_df_copy
    
#     # convert to lists
#     expand_df['subjects'] = expand_df['subjects'].str.findall(f"'([\w\./]+)'")
#     expand_df['audiences'] = expand_df['audiences'].str.findall(f"'([\w\./]+)'")
    
    # clean headlines
    expand_df['headline'] = expand_df['headline'].apply(clean_headlines)
    
    # group categoricals
    expand_cat_df = categorical_groupby(expand_df)
    expand_cat_df['time'] = expand_df['time']
    expand_cat_df['assetCode'] = expand_df['assetCode']
    
#     # convert to sets
#     for cat_col in ['subjects', 'audiences']:
#         expand_cat_df[cat_col] = expand_cat_df[cat_col].apply(tuple)
        
    # remove duplicate rows
    expand_cat_df = expand_cat_df.drop_duplicates(subset=['time', 'assetCode'],
                                                  keep='first')
    
    # merge datasets
    expanded_market_df = pd.merge(market_df_copy, expand_cat_df, 
                                  on=['time', 'assetCode'], how='left')
    
    return expanded_market_df
    

In [19]:
%%time
X_train = merge_by_code(market_train_df, expand_train_df)

CPU times: user 3.88 s, sys: 1.32 s, total: 5.2 s
Wall time: 5.2 s


In [20]:
X_train.tail()

,time,assetCode,volume,close,open,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,headline
892195,2016-12-30 22:00:00+00:00,ZIOP.O,1608829.0,5.35,5.37,-0.165367,-0.138042,-0.139597,-0.135913,0.051189,NaN
892196,2016-12-30 22:00:00+00:00,ZLTQ.O,347830.0,43.52,43.62,0.002996,0.002989,0.008213,0.003210,-0.048555,NaN
892197,2016-12-30 22:00:00+00:00,ZNGA.O,7396601.0,2.57,2.58,-0.091873,-0.078571,-0.077252,-0.077188,0.011703,NaN
892198,2016-12-30 22:00:00+00:00,ZTO.N,3146519.0,12.07,12.50,-0.065066,-0.042146,-0.078104,-0.043813,0.083367,NaN
892199,2016-12-30 22:00:00+00:00,ZTS.N,1701204.0,53.53,53.64,0.023127,0.028177,0.026566,0.028719,-0.016220,NaN


In [21]:
def data_step1(market_test_df, news_test_df):
    
    market_test_df, news_test_df = clean_data(market_test_df, news_test_df, train=False)
    expand_test_df = expanding_news(news_test_df)
    X_test = merge_by_code(market_test_df, expand_test_df)
    del market_test_df, news_test_df, expand_test_df
    gc.collect()
    
    return X_test
    

In [22]:
del market_train_df, news_train_df, expand_train_df
gc.collect()

96

<a id='section2'></a>
## Step 2. Feature Engineering

From Quant features to text processing features.

In [23]:
X_train.tail()

,time,assetCode,volume,close,open,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,headline
892195,2016-12-30 22:00:00+00:00,ZIOP.O,1608829.0,5.35,5.37,-0.165367,-0.138042,-0.139597,-0.135913,0.051189,NaN
892196,2016-12-30 22:00:00+00:00,ZLTQ.O,347830.0,43.52,43.62,0.002996,0.002989,0.008213,0.003210,-0.048555,NaN
892197,2016-12-30 22:00:00+00:00,ZNGA.O,7396601.0,2.57,2.58,-0.091873,-0.078571,-0.077252,-0.077188,0.011703,NaN
892198,2016-12-30 22:00:00+00:00,ZTO.N,3146519.0,12.07,12.50,-0.065066,-0.042146,-0.078104,-0.043813,0.083367,NaN
892199,2016-12-30 22:00:00+00:00,ZTS.N,1701204.0,53.53,53.64,0.023127,0.028177,0.026566,0.028719,-0.016220,NaN


### Text Processing with CountVectorizer
We are going to be using CountVectorizer on headlines, audiences and subjects to determine its influence on the target column.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

class TextToCoeff:

    def vectorize(self, X_):

        # get required dataset
        X_text = X_.copy()[['returnsOpenNextMktres10', self.col]].dropna()

        # round data (if objective : binary)
        def round_scores(x):
            if x >= 0:
                return 1
            else:
                return 0
        X_text['returnsOpenNextMktres10'] = X_text['returnsOpenNextMktres10'].apply(round_scores)

        # convert tuples to string format if applicable
        def tuple_to_str(x):
            try:
                if isinstance(x, tuple):
                    return ' '.join(x)
                else:
                    return x
            except ValueError:
                return np.nan
            
        X_text[self.col] = X_text[self.col].apply(tuple_to_str)

        # get lists
        text_lst = list(X_text[self.col])
        target_lst = list(X_text['returnsOpenNextMktres10'])

        # vectorize text features
        vectorizer = CountVectorizer()
        text_vectorized = vectorizer.fit_transform(text_lst)

        # model data (will use other modelling methods in the future)
        text_model = LogisticRegression()
        text_model = text_model.fit(text_vectorized, target_lst)

        # get coefficients
        basictext = vectorizer.get_feature_names()
        basiccoeffs = text_model.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Text' : basictext, 
                                'Coefficient' : basiccoeffs})

        # convert dataframe to dictionary of coefficients
        self.coeff_dict = dict(zip(coeff_df.Text, coeff_df.Coefficient))

        # get value that accounts for nulls
        self.coeff_default = coeff_df['Coefficient'].mean()

    def predict_data(self, X_):

        def get_coeff(x):
            
            try:

                # iterate through each set of text data
                coeff_total_score = 0
                
                if isinstance(x, tuple):
                    x = ' '.join(x)
                    
                if isinstance(x, str):
                    x = [x]
                
                for textset in x:

                    text_lst = textset.split(' ')
                
                    # iter through every word
                    coeff_sum = 0
                    for text in text_lst:
                        text = text.lower()
                        if text in self.coeff_dict:
                            coeff_sum += self.coeff_dict[text]
                        else:
                            coeff_sum += self.coeff_default

                    # get average coefficient
                    coeff_total_score += coeff_sum / len(text_lst)

                return coeff_total_score / len(x)
            
            except TypeError:
                
                return np.nan

        X_[self.col + '_coeff_mean'] = X_[self.col].apply(get_coeff)
        
        return X_

    # obtain target volumn
    def __init__(self, input_col):
        self.col = input_col

In [25]:
def text_processing_train(X):
    
    # get list of text to coeff converters [headline, subjects, audiences]
    text_to_coeff_lst = []
#     for f in ['headline', 'subjects', 'audiences']:
    for f in ['headline']:
        text_to_coeff = TextToCoeff(f)
        text_to_coeff.vectorize(X)
        text_to_coeff_lst.append(text_to_coeff)
    
    return text_to_coeff_lst

def text_processing_test(X, text_to_coeff_lst):
    
    for text_to_coeff in text_to_coeff_lst:
        X = text_to_coeff.predict_data(X)
        
    del X['headline']
#     del X['subjects']
#     del X['audiences']
    gc.collect()
    
    return X

In [26]:
%%time
text_to_coeff_lst = text_processing_train(X_train)
X_train = text_processing_test(X_train, text_to_coeff_lst)

CPU times: user 1.5 s, sys: 4 ms, total: 1.5 s
Wall time: 1.51 s


### Entire Market and Individual Asset Lag Features
We are going to be obtaining Quant Features from both the entire market dataframe and from each individual asset based on assetCode.

Source: https://www.kaggle.com/qqgeogor/eda-script-67

In [27]:
from multiprocessing import Pool

def create_lag(df_code,n_lag=[3,7,14,],shift_size=1):
    code = df_code['assetCode'].unique()
    
    for col in return_features:
        for window in n_lag:
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            lag_mean = rolled.mean()
            lag_max = rolled.max()
            lag_min = rolled.min()
            lag_std = rolled.std()
            df_code['%s_lag_%s_mean'%(col,window)] = lag_mean
            df_code['%s_lag_%s_max'%(col,window)] = lag_max
            df_code['%s_lag_%s_min'%(col,window)] = lag_min

    return df_code.fillna(-1)

def generate_lag_features(df,n_lag = [3,7,14]):
    
    assetCodes = df['assetCode'].unique()
    all_df = []
    df_codes = df.groupby('assetCode')
    df_codes = [df_code[1][['time','assetCode']+return_features] for df_code in df_codes]
    
    pool = Pool(4)
    all_df = pool.map(create_lag, df_codes)
    
    new_df = pd.concat(all_df)  
    new_df.drop(return_features+['time', 'assetCode'],axis=1,inplace=True)
    new_df = pd.concat([df, new_df], axis=1, sort=False)
    pool.close()
    
    return new_df

In [28]:
%%time
return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close', 'volume']
X_train = generate_lag_features(X_train)

CPU times: user 7.54 s, sys: 3.77 s, total: 11.3 s
Wall time: 45.3 s


In [29]:
X_train.dropna().head()

,time,assetCode,volume,close,open,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,headline_coeff_mean,returnsClosePrevMktres10_lag_3_mean,returnsClosePrevMktres10_lag_3_max,returnsClosePrevMktres10_lag_3_min,returnsClosePrevMktres10_lag_7_mean,returnsClosePrevMktres10_lag_7_max,returnsClosePrevMktres10_lag_7_min,returnsClosePrevMktres10_lag_14_mean,returnsClosePrevMktres10_lag_14_max,returnsClosePrevMktres10_lag_14_min,returnsClosePrevRaw10_lag_3_mean,returnsClosePrevRaw10_lag_3_max,returnsClosePrevRaw10_lag_3_min,returnsClosePrevRaw10_lag_7_mean,returnsClosePrevRaw10_lag_7_max,returnsClosePrevRaw10_lag_7_min,returnsClosePrevRaw10_lag_14_mean,returnsClosePrevRaw10_lag_14_max,returnsClosePrevRaw10_lag_14_min,open_lag_3_mean,open_lag_3_max,open_lag_3_min,open_lag_7_mean,open_lag_7_max,open_lag_7_min,open_lag_14_mean,open_lag_14_max,open_lag_14_min,close_lag_3_mean,close_lag_3_max,close_lag_3_min,close_lag_7_mean,close_lag_7_max,close_lag_7_min,close_lag_14_mean,close_lag_14_max,close_lag_14_min,volume_lag_3_mean,volume_lag_3_max,volume_lag_3_min,volume_lag_7_mean,volume_lag_7_max,volume_lag_7_min,volume_lag_14_mean,volume_lag_14_max,volume_lag_14_min
3522,2015-01-06 22:00:00+00:00,BERY.N,1773553.0,31.82,31.85,0.042937,0.034427,0.052021,0.040911,0.033266,0.052720,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4811,2015-01-06 22:00:00+00:00,TRMK.O,459378.0,23.06,23.63,-0.063363,-0.022342,-0.029527,-0.004954,-0.072428,-0.029931,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000e+00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5424,2015-01-07 22:00:00+00:00,DB.N,2689385.0,28.66,28.61,-0.070084,-0.071405,-0.045118,-0.043554,0.021101,0.087410,-0.041690,-0.021221,-0.061425,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.052338,0.001973,-0.082166,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,29.636667,30.59,28.96,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,29.216667,30.47,28.36,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.787095e+06,3302828.0,2273233.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5836,2015-01-07 22:00:00+00:00,KND.N,1846453.0,17.83,17.16,-0.029924,-0.047196,-0.006754,-0.016487,0.085713,0.095380,-0.040549,-0.020379,-0.053088,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.049903,-0.031080,-0.067633,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,17.800000,18.34,17.37,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,17.420000,17.77,17.12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.524299e+06,2014114.0,959873.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6557,2015-01-07 22:00:00+00:00,VIAB.O,3639112.0,71.30,72.35,-0.066510,-0.047525,-0.037256,-0.014685,-0.077258,-0.061035,-0.008926,0.002563,-0.017463,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.020317,0.024874,-0.053609,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,74.696667,75.83,73.46,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,73.516667,75.40,71.85,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.896212e+06,3568991.0,1976737.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


### Misc. Features
Inclues the following features:
* Daily Difference

In [30]:
def misc_features(X):
    
    # Adding daily difference
    new_col = X["close"] - X["open"]
    X.insert(loc=6, column="daily_diff", value=new_col)
    X['close_to_open'] =  np.abs(X['close'] / X['open'])
    
    # extra features
    X['bartrend'] = X['close'] / X['open']
    X['average'] = (X['close'] + X['open'])/2
    X['pricevolume'] = X['volume'] * X['close']


In [31]:
misc_features(X_train)

In [32]:
X_train.dropna().tail()

,time,assetCode,volume,close,open,returnsClosePrevRaw10,daily_diff,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,headline_coeff_mean,returnsClosePrevMktres10_lag_3_mean,returnsClosePrevMktres10_lag_3_max,returnsClosePrevMktres10_lag_3_min,returnsClosePrevMktres10_lag_7_mean,returnsClosePrevMktres10_lag_7_max,returnsClosePrevMktres10_lag_7_min,returnsClosePrevMktres10_lag_14_mean,returnsClosePrevMktres10_lag_14_max,returnsClosePrevMktres10_lag_14_min,returnsClosePrevRaw10_lag_3_mean,returnsClosePrevRaw10_lag_3_max,returnsClosePrevRaw10_lag_3_min,returnsClosePrevRaw10_lag_7_mean,returnsClosePrevRaw10_lag_7_max,returnsClosePrevRaw10_lag_7_min,returnsClosePrevRaw10_lag_14_mean,returnsClosePrevRaw10_lag_14_max,returnsClosePrevRaw10_lag_14_min,open_lag_3_mean,open_lag_3_max,open_lag_3_min,open_lag_7_mean,open_lag_7_max,open_lag_7_min,open_lag_14_mean,open_lag_14_max,open_lag_14_min,close_lag_3_mean,close_lag_3_max,close_lag_3_min,close_lag_7_mean,close_lag_7_max,close_lag_7_min,close_lag_14_mean,close_lag_14_max,close_lag_14_min,volume_lag_3_mean,volume_lag_3_max,volume_lag_3_min,volume_lag_7_mean,volume_lag_7_max,volume_lag_7_min,volume_lag_14_mean,volume_lag_14_max,volume_lag_14_min,close_to_open,bartrend,average,pricevolume
880118,2016-12-21 22:00:00+00:00,GOOG.O,1211346.0,794.56,795.84,0.030304,-1.28,0.045782,0.008581,-0.002074,0.009318,0.145861,0.015027,0.030231,0.000888,0.026553,0.049086,0.000888,0.011370,0.049086,-0.028080,0.048131,0.053698,0.041546,0.046112,0.066758,0.027374,0.023147,0.066758,-0.026879,795.793333,800.40,790.22,794.437143,800.40,785.04,778.500714,800.40,744.59,793.806667,796.42,790.80,794.530000,797.85,789.27,779.904286,797.85,747.92,1.542299e+06,2443796.0,951014.0,1.743839e+06,2443796.0,951014.0,1.773797e+06,3017947.0,951014.0,0.998392,0.998392,795.200,9.624871e+08
880119,2016-12-21 22:00:00+00:00,GOOGL.O,1459628.0,812.20,815.72,0.026192,-3.52,0.045862,0.004218,-0.000078,0.013145,0.145861,0.020601,0.039960,0.003058,0.032340,0.054876,0.003058,0.015764,0.054876,-0.028711,0.051228,0.059362,0.044049,0.047988,0.067144,0.028137,0.024758,0.067144,-0.027603,813.653333,818.31,809.28,813.064286,818.31,804.82,796.735000,818.31,761.90,812.513333,815.20,809.84,813.474286,817.89,807.90,798.114286,817.89,764.33,1.711443e+06,2598866.0,1263581.0,1.780074e+06,2598866.0,1263581.0,1.858244e+06,2867074.0,1263581.0,0.995685,0.995685,813.960,1.185510e+09
880182,2016-12-21 22:00:00+00:00,HMC.N,507281.0,30.02,30.08,0.000000,-0.06,0.013136,0.002266,0.024439,0.006083,0.145861,0.034245,0.035138,0.033067,0.033763,0.041578,0.021137,0.033442,0.056021,0.012214,0.024689,0.026164,0.022026,0.023488,0.031762,0.009079,0.030351,0.057183,0.009079,30.146667,30.21,30.10,30.160000,30.21,30.09,29.937857,30.34,29.31,30.163333,30.20,30.13,30.171429,30.30,30.01,30.007143,30.63,29.37,5.667863e+05,854758.0,391865.0,6.697453e+05,1054597.0,391865.0,7.215958e+05,1054597.0,391865.0,0.998005,0.998005,30.050,1.522858e+07
882475,2016-12-22 22:00:00+00:00,PBH.N,363975.0,48.74,49.14,0.047496,-0.40,0.050224,0.023523,0.008091,0.070904,0.134257,-0.043596,0.005349,-0.076473,-0.055516,0.005349,-0.095553,-0.075657,0.005349,-0.118269,0.056050,0.059417,0.053082,0.055421,0.079802,0.023899,0.004650,0.079802,-0.078879,49.586667,49.70,49.51,49.475714,50.42,48.07,48.047857,50.42,46.35,49.303333,49.39,49.20,49.534286,50.20,49.20,48.198571,50.20,46.36,3.317723e+05,358254.0,292139.0,5.680126e+05,1416838.0,292139.0,5.008767e+05,1416838.0,292139.0,0.991860,0.991860,48.940,1.774014e+07
883039,2016-12-22 22:00:00+00:00,XLRN.O,580303.0,26.26,27.15,-0.165820,-0.89,-0.124194,-0.147845,-0.113162,0.032059,0.078197,-0.242522,-0.109602,-0.369297,-0.426109,-0.109602,-0.632957,-0.384752,-0.068863,-0.632957,-0.152812,-0.112080,-0.174785,-0.163599,-0.112080,-0.190307,-0.154126,-0.048755,-0.218009,28.946667,29.88,28.13,29.047143,29.88,28.13,31.130714,37.22,28.13,28.146667,28.80,27.49,28.800000,29.66,27.49,30.529286,36.29,27.49,2.629113e+05,293882.0

In [33]:
def data_step2(X_test):
    
    X_test = generate_lag_features(X_test)
    X_test = text_processing_test(X_test, text_to_coeff_lst)
    misc_features(X_test)
    
    return X_test

<a id='section3'></a>
## Step 3. Modelling using LightGBM

In [34]:
traincols = [f for f in X_train.columns if f not in ['time', 'assetCode', 'universe', 'assetName', \
                                                     'returnsOpenNextMktres10','headline']]

# classify data
y_train = (X_train['returnsOpenNextMktres10'] > 0).astype(int)
X_train = X_train[traincols].fillna(0)

### Fixed Training Split
The reason why we need to do a fixed training test split that fetches the last few rows of the training dataset is to avoid odd results, since randomly choosing rows will cause the validation dataset to be filled with rows with different timestamps.

In [35]:
def fixed_train_test_split(X, y, train_size):
    
    # round train size
    train_size = int(train_size * len(X))
    
    # split data
    X_train, y_train = X[train_size:].values, y[train_size:].values
    X_valid, y_valid = X[:train_size].values, y[:train_size].values
    
    return X_train, y_train, X_valid, y_valid

In [36]:
X_train, y_train, X_valid, y_valid = fixed_train_test_split(X_train, y_train, 0.85)

In [37]:
# Scaling of X values
# It is good to keep these scaling values for later
mins = np.min(X_train, axis=0)
maxs = np.max(X_train, axis=0)
rng = maxs - mins
X_train = 1 - ((maxs - X_train) / rng)

### Using LightGBM for modelling

Models from https://www.kaggle.com/qqgeogor/eda-script-67

In [38]:
%%time

import lightgbm as lgb

# these are tuned params I found
x_1 = [0.19000424246380565, 2452, 212, 328, 202]
x_2 = [0.19016805202090095, 2583, 213, 312, 220]

train_data, test_data = lgb.Dataset(X_train, y_train), lgb.Dataset(X_valid, y_valid)

params_1 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        'learning_rate': x_1[0],
        'num_leaves': x_1[1],
        'min_data_in_leaf': x_1[2],
#         'num_iteration': x_1[3],
        'num_iteration': 239,
        'max_bin': x_1[4],
        'verbose': 1
    }

params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        'learning_rate': x_2[0],
        'num_leaves': x_2[1],
        'min_data_in_leaf': x_2[2],
#         'num_iteration': x_2[3],
        'num_iteration': 172,
        'max_bin': x_2[4],
        'verbose': 1
    }

gbm_1 = lgb.train(params_1,
        train_data,
        num_boost_round=100,
        valid_sets=test_data,
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )

gbm_2 = lgb.train(params_2,
        train_data,
        num_boost_round=100,
        valid_sets=test_data,
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )


[1]	valid_0's binary_logloss: 0.70505
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.72542
[3]	valid_0's binary_logloss: 0.756944
[4]	valid_0's binary_logloss: 0.792997
[5]	valid_0's binary_logloss: 0.835192
[6]	valid_0's binary_logloss: 0.856446
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.70505
[1]	valid_0's binary_logloss: 0.705068
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.724581
[3]	valid_0's binary_logloss: 0.757593
[4]	valid_0's binary_logloss: 0.798879
[5]	valid_0's binary_logloss: 0.811772
[6]	valid_0's binary_logloss: 0.854176
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.705068
CPU times: user 12.2 s, sys: 1.15 s, total: 13.4 s
Wall time: 3.98 s


<a id='section4'></a>
## Step 4. Applying the Model


Predictions will be made through a for loop, and apply all the functions above onto the test dataset.

In [39]:
def get_X(market_test_df, news_test_df):
    X_test = data_step1(market_test_df, news_test_df)
    X_test = data_step2(X_test)
    X_test = X_test[[f for f in X_test.columns if f not in ['time', 'assetCode', 'universe', 'assetName', 'returnsOpenNextMktres10',
                                                          'headline', 'subjects', 'audiences']]].fillna(0).values
    return X_test

def make_predictions(market_obs_df, news_obs_df):
    
    # predict using given model
    X_test = get_X(market_obs_df, news_obs_df)
    X_test = 1 - ((maxs - X_test) / rng)
    prediction_values = (gbm_1.predict(X_test) + gbm_2.predict(X_test))/2
    prediction_values = (prediction_values-prediction_values.min())/(prediction_values.max()-prediction_values.min())
    prediction_values = prediction_values * 2 - 1

    return prediction_values


In [40]:
%%time
n_days = 0
for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days(): # Looping over days from start of 2017 to 2019-07-15
    
    n_days +=1
    if n_days % 50 == 0:
        print(n_days)
    
    # make predictions
    predictions_template_df['confidenceValue'] = make_predictions(market_obs_df, news_obs_df)
    
    # save predictions
    env.predict(predictions_template_df)
    
env.write_submission_file()

1.0 -1.0
1.0 -1.0
1.0 -1.0
1.0 -1.0
1.0 -1.0
1.0 -1.0


Process ForkPoolWorker-30:
Process ForkPoolWorker-31:
Process ForkPoolWorker-29:
Process ForkPoolWorker-32:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-27-8a324a7b1665>", line 10, in create_lag
    lag_max = rolled.max()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/window.py", line 1593, in max
    return super(Rolling, self).max(*args, **kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line

KeyboardInterrupt: 

**Sources:**
* [Market Data NN Baseline by Christofhenkel](https://www.kaggle.com/christofhenkel/market-data-nn-baseline)
* [A simple model using the market and news data by Bguberfain](https://www.kaggle.com/bguberfain/a-simple-model-using-the-market-and-news-data)
* [Amateur Hour - Using Headlines to Predict Stocks by Magichanics](https://www.kaggle.com/magichanics/amateur-hour-using-headlines-to-predict-stocks)
* [Simple Quant Features by Youhanlee](https://www.kaggle.com/youhanlee/simple-quant-features-using-python)
* [>0.64 in 100 lines by rabaman](https://www.kaggle.com/rabaman/0-64-in-100-lines/comments)
* [eda script 67 by qqgeogor](https://www.kaggle.com/qqgeogor/eda-script-67)